## Functional data structures
Functional data structures are by definition immutable. As an introduction, consider singly linked list.


In [1]:
sealed trait List[+A] // Variance annotation: see below
case object Nil extends List[Nothing] // Constructor for empty list. `Nothing` is a subtype of all types.
case class Cons[+A](head: A, tail: List[A]) extends List[A] // Constructor for non-empty list

object List { // Companion object containing useful tools
    def sum(ints: List[Int]): Int = ints match { // Pattern matching
        case Nil => 0
        case Cons(x, xs) => x + sum(xs) // Capture `x` and `xs`, call `sum` recursively
    }
    
    def product(ds: List[Double]): Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x, xs) => x * product(xs)
    }
    
    def apply[A](as: A*): List[A] = { // Variadic function syntax (variable number of arguments)
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }
}

defined trait List
defined object Nil
defined class Cons
defined object List

The syntax 
```scala
List[+A]
```
means above that `A` is a covariant ("positive") parameter of `List`. For example, `List[Dog]` is now a subtype of `List[Animal]` if `Dog` is a subtype of `Animal`. More generally, the variance annotation means if `X` is a subtype of `Y`, then `List[X]` is a subtype of `List[Y]`.

### Exercise 3.1
Pattern matching example

In [2]:
val x = List(1, 2, 3, 4, 5) match {
    case Cons(x, Cons(2, Cons(4, _))) => x // Does not match
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4, _)))) => x + y // Matches, returns 1+2
    case Cons(h, t) => h + List.sum(t) // Also matches but not the first
    case _ => 101
}

assert(x == 3)

x: Int = 3

### Exercise 3.2
Data sharing in functional data structures: Implement function `tail` for removing the first element of a `List`. Note that the input `List` is _not_ modified.

In [3]:
def tail[A](as: List[A]): List[A] = {
    as match {
        case Nil => Nil // There are different choices for empty List that one could do here.
        case Cons(h, t) => t // Return the tail. `t` is immutable so no copying is needed.
    }
}

tail(Nil) // Nil
tail(List(1)) // Nil
tail(List(1, 2)) // Cons(2, Nil)

defined function tail
res2_1: List[Nothing] = Nil
res2_2: List[Int] = Nil
res2_3: List[Int] = Cons(2,Nil)

### Exercise 3.3
Implement `setHead` for replacing the first element of a List.

In [4]:
def setHead[A](a: A, as: List[A]): List[A] = {
    as match {
        case Nil => Cons(a, Nil)
        case Cons(h, t) => Cons(a, t) // Data sharing: no need to copy `t` as it is immutable.
    }
}

setHead(3, Nil) // Cons(3, Nil)
setHead(3, List(1, 2)) // Cons(3, Cons(2, Nil))

defined function setHead
res3_1: List[Int] = Cons(3,Nil)
res3_2: List[Int] = Cons(3,Cons(2,Nil))

### Exercise 3.4
Define function `drop` that removes the first `n` elements from a list.

In [5]:
@annotation.tailrec
def drop[A](l: List[A], n: Int): List[A] = {
    if (n <= 0) l
    else l match {
        case Nil => Nil
        case Cons(h, t) => drop(t, n-1)
    }
}

val l = List(1, 2, 3, 4)

drop(l, 1)
drop(l, 2)
drop(l, 3)
drop(l, 4)
drop(l, 5)

defined function drop
l: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Nil))))
res4_2: List[Int] = Cons(2,Cons(3,Cons(4,Nil)))
res4_3: List[Int] = Cons(3,Cons(4,Nil))
res4_4: List[Int] = Cons(4,Nil)
res4_5: List[Int] = Nil
res4_6: List[Int] = Nil

### Exercise 3.5
Implement `dropWhile` that removes elements from the `List` as long as they match a predicate.

In [6]:
@annotation.tailrec
def dropWhile[A](l: List[A], f: A => Boolean): List[A] = {
  l match {
      case Nil => Nil
      case Cons(h, t) => if (f(h)) dropWhile(t, f) else l
  }  
}

val l = List(1, 2, 3, 4, 7, 5)
dropWhile(l, (x: Int) => x < 5) // Cons(7, Cons(5, _))
dropWhile(l, (x: Int) => x < 8) // Nil
dropWhile(l, (x: Int) => x < 0) // l

defined function dropWhile
l: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(7,Cons(5,Nil))))))
res5_2: List[Int] = Cons(7,Cons(5,Nil))
res5_3: List[Int] = Nil
res5_4: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(7,Cons(5,Nil))))))

It's a bit annoying that the function `f` requires type argument even though it should be obvious from the `List` type – this does not compile:
```scala
dropWhile(l, x => x < 5)
```
To fix this, one can give curried definition that improves type inference:

In [7]:
@annotation.tailrec
def dropWhileCurried[A](l: List[A])(f: A => Boolean): List[A] = {
    l match {
        case Cons(h, t) => if (f(h)) dropWhileCurried(t)(f) else l
        case _ => l
    }
}

dropWhileCurried(l)(x => x < 5) // Works!

defined function dropWhileCurried
res6_1: List[Int] = Cons(7,Cons(5,Nil))

### Exercise 3.6
Implement a function `init` that returns a `List` consisting of all but the last element of a `List`. Note that whereas `tail` takes constant time, this takes linear time.

In [8]:
def init[A](l: List[A]): List[A] = {
    l match {
        case Nil => Nil
        case Cons(h, Nil) => Nil
        case Cons(h, t) => Cons(h, init(t)) // This essentially copies the whole list.
    }
}

init(List(1, 2, 3, 4)) // Cons(1, Cons(2, Cons(3, Nil))), i.e. List(1, 2, 3)

defined function init
res7_1: List[Int] = Cons(1,Cons(2,Cons(3,Nil)))

### Generalizing recursion to higher-order functions
`sum` and `product` defined in `List` companion object can be defined using a common `foldRight` operator:

In [9]:
def foldRight[A, B](as: List[A], z: B)(f: (A, B) => B): B = 
    as match {
        case Nil => z
        case Cons(h, t) => f(h, foldRight(t, z)(f)) // Push frames onto the call stack all the way until at the end
    }

def sum2(ns: List[Int]): Int = 
    foldRight(ns, 0)(_ + _) // Don't worry about the empty list for now

def product2(ds: List[Double]): Double = 
    ds match {
        case Nil => throw new Error("Empty list")
        case _ => foldRight(ds, 1.0)(_*_)
    }
    
sum2(List(1, 2, 3, 4, 5))
product2(List(1, 2, 3, 4, 5))

defined function foldRight
defined function sum2
defined function product2
res8_3: Int = 15
res8_4: Double = 120.0

### Exercise 3.9
Compute the length of a list using `foldRight`.

In [10]:
def length[A](as: List[A]): Int = {
    foldRight(as, 0)((_, b) => b + 1)
}

assert(length(List(1, 3, 2, 4)) == 4)

defined function length

### Exercise 3.10
`foldRight` is not tail-recursive and is not stack-safe. Write `foldLeft` that is tail-recursive.

In [11]:
@annotation.tailrec
def foldLeft[A, B](as: List[A], z: B)(f: (B, A) => B): B =
    as match {
        case Nil => z
        case Cons(h, t) => foldLeft(t, f(z, h))(f)
    }

def sum3(ns: List[Int]): Int = 
    foldLeft(ns, 0)(_ + _)

assert(sum3(List(1, 3, 5, 7)) == 16)

defined function foldLeft
defined function sum3

### Exercise 3.12
Write a function that returns the reverse of a list.

In [12]:
/* First try with append */
def append[A](as: List[A], x: A): List[A] = {
    as match {
        case Nil => Cons(x, Nil)
        case Cons(h, t) => Cons(h, append(t, x))
    }
}

def reverse[A](as: List[A]): List[A] = {
    as match {
        case Nil => Nil
        case Cons(h, t) => append(reverse(t), h)
    }
}

val l = List(1, 2, 3, 4, 5)

reverse(l) // Works

// Second try with `append` and `foldRight`
foldRight(l, Nil: List[Int])((a, z) => append(z, a))

// Final version with `foldLeft`, place new elements to the beginning when traversing from the left
foldLeft(l, Nil: List[Int])((z, a) => Cons(a, z))

defined function append
defined function reverse
l: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))
res11_3: List[Int] = Cons(5,Cons(4,Cons(3,Cons(2,Cons(1,Nil)))))
res11_4: List[Int] = Cons(5,Cons(4,Cons(3,Cons(2,Cons(1,Nil)))))
res11_5: List[Int] = Cons(5,Cons(4,Cons(3,Cons(2,Cons(1,Nil)))))

### Exercise 3.14
Implement `append` in terms of either `foldLeft` or `foldRight`.

In [13]:
def append[A](as: List[A], a: A): List[A] = foldRight(as, Cons(a, Nil))((b, z) => Cons(b, z))

append(List(1, 2, 3), 4)

defined function append
res12_1: List[Int] = Cons(1,Cons(2,Cons(3,Cons(4,Nil))))

### Exercise 3.16
Write a function that transforms a list of integers by adding `1` to each element.

In [14]:
def addOne(ns: List[Int]): List[Int] = foldRight(ns, Nil: List[Int])((a, z) => Cons(a+1, z))

addOne(List(1, 2, 3, 4, 5))

defined function addOne
res13_1: List[Int] = Cons(2,Cons(3,Cons(4,Cons(5,Cons(6,Nil)))))

### Exercise 3.17
Write a function that transforms each value in `List[Double]` to `String`.

In [15]:
def toString(ds: List[Double]): List[String] = foldRight(ds, Nil: List[String])((a, z) => Cons(s"Value is ${a}", z))
toString(List(1.0, 2.0, 3.0))

defined function toString
res14_1: List[String] = Cons(Value is 1.0,Cons(Value is 2.0,Cons(Value is 3.0,Nil)))

### Exercise 3.18
Write `map` function.

In [16]:
def map[A, B](as: List[A])(f: A => B): List[B] = 
    foldRight(as, Nil: List[B])((a, z) => Cons(f(a), z))

map(List(1.0, 2.0, 3.0))(a => s"Mapped to ${a}")

defined function map
res15_1: List[String] = Cons(Mapped to 1.0,Cons(Mapped to 2.0,Cons(Mapped to 3.0,Nil)))

### Exercise 3.19
Write function `filter`.

In [17]:
def filter[A](as: List[A])(f: A => Boolean) =
    foldRight(as, Nil: List[A])((a, z) => if (f(a)) Cons(a, z) else z)

assert(filter(List(1, 2, 3, 4))(x => x != 3) == List(1, 2, 4))

defined function filter

### Exercise 3.20
Write function `flatMap`.

In [18]:
def concatenate[A](as1: List[A], as2: List[A]): List[A] =
    foldRight(as1, as2)((a, z) => Cons(a, z))

def flatMap[A, B](as: List[A])(f: A => List[B]): List[B] =
    foldRight(as, Nil: List[B])((a, z) => concatenate(f(a), z))

assert(flatMap(List(1, 2))(a => List(a, a)) == List(1, 1, 2, 2))

defined function concatenate
defined function flatMap

### Exercise 3.21
Use `flatMap` to implement `filter`.

In [19]:
def filter[A](as: List[A])(f: A => Boolean): List[A] = 
    flatMap(as)(a => if (f(a)) List(a) else Nil)

filter(List(1, 2, 3, 4))(x => x != 3)

defined function filter
res18_1: List[Int] = Cons(1,Cons(2,Cons(4,Nil)))

### Exercise 3.23
Write function `zipWith`.

In [20]:
def zipWith[A, B, C](a1: List[A], a2: List[B])(f: (A, B) => C): List[C] =
    (a1, a2) match {
        case (Nil, Nil) => Nil
        case (Cons(h1, t1), Cons(h2, t2)) => Cons(f(h1, h2), zipWith(t1, t2)(f)) // Expect same length
    }

zipWith(List(1, 2, 3), List(4, 5, 6))(_ + _)

defined function zipWith
res19_1: List[Int] = Cons(5,Cons(7,Cons(9,Nil)))

## Trees
As another example of a functional data structure, consider binary tree. Note that in this definition, branches do not contain any values, only leafs do.

In [21]:
sealed trait Tree[+A]
case class Leaf[A](value: A) extends Tree[A]
case class Branch[A](left: Tree[A], right: Tree[A]) extends Tree[A]

defined trait Tree
defined class Leaf
defined class Branch

### Exercises 3.25-3.28
Write functions `size`, `maximum`, `depth`, and `map` for `Tree`.

In [22]:
def size[A](t: Tree[A]): Int = {
    t match {
        case Leaf(v) => 1
        case Branch(left, right) => 1 + size(left) + size(right)
    }
}

val leftTree = new Branch(left = new Branch(left = new Leaf(1), right = new Leaf(5)), right = new Leaf(4))
val rightTree = new Branch(left = new Leaf(3), right = new Leaf(2))
val tree = new Branch(left = leftTree, right = rightTree)

assert(size(tree) == 9)

defined function size
leftTree: Branch[Int] = Branch(Branch(Leaf(1),Leaf(5)),Leaf(4))
rightTree: Branch[Int] = Branch(Leaf(3),Leaf(2))
tree: Branch[Int] = Branch(Branch(Branch(Leaf(1),Leaf(5)),Leaf(4)),Branch(Leaf(3),Leaf(2)))

In [23]:
def maximum(t: Tree[Int]): Int = 
    t match {
        case Leaf(v) => v
        case Branch(left, right) => maximum(left) max maximum(right)
    }

assert(maximum(tree) == 5)

defined function maximum

In [24]:
def depth[A](t: Tree[A]): Int = 
    t match {
        case Leaf(v) => 0
        case Branch(left, right) => 1 + depth(left) max depth(right)
    }

assert(depth(tree) == 3)

defined function depth

In [25]:
def map[A,B](t: Tree[A])(f: A => B): Tree[B] = 
    t match {
        case Leaf(v) => new Leaf(f(v))
        case Branch(left, right) => new Branch(left = map(left)(f), right = map(right)(f))
    }

map(tree)(v => s"Value is ${v}")

defined function map
res24_1: Tree[String] = Branch(Branch(Branch(Leaf(Value is 1),Leaf(Value is 5)),Leaf(Value is 4)),Branch(Leaf(Value is 3),Leaf(Value is 2)))

### Exercise 3.29
Generalize the functions defined above by writing a new function `fold`.

In [26]:
/**
 * @param t Tree
 * @param f Function applied to values in the leaves
 * @param combine Function applied to combine values from two leaves
 * @return Value from the fold
**/
def fold[A,B](t: Tree[A])(f: A => B)(combine: (B, B) => B): B = 
    t match {
        case Leaf(v) => f(v)
        case Branch(left, right) => combine(fold(left)(f)(combine), fold(right)(f)(combine))
    }

def size[A](t: Tree[A]): Int = 
    fold(t)((l) => 1)((left, right) => 1 + left + right)

def maximum(t: Tree[Int]): Int =
    fold(t)((l) => l)((left, right) => left max right)

def depth[A](t: Tree[A]): Int = 
    fold(t)((l) => 0)((left, right) => 1 + left max right)

// Map is a tad trickier: mapping function given to `fold` needs to return a tree so that the 
// types match
def map[A, B](t: Tree[A])(f: A => B): Tree[B] = 
    fold(t)(v => new Leaf(f(v)): Tree[B])((left, right) => new Branch(left, right))

assert(size(tree) == 9)
assert(maximum(tree) == 5)
assert(depth(tree) == 3)
map(tree)(v => s"Value is ${v}")

defined function fold
defined function size
defined function maximum
defined function depth
defined function map
res25_8: Tree[String] = Branch(Branch(Branch(Leaf(Value is 1),Leaf(Value is 5)),Leaf(Value is 4)),Branch(Leaf(Value is 3),Leaf(Value is 2)))